In [1]:
import pandas as pd
from sqlalchemy import create_engine
from secret import username, password

In [2]:
#import csv - Athletes

athlete_file = "../Resources/athlete_events.csv"
athlete_data_df = pd.read_csv(athlete_file)
athlete_data_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [3]:
#import csv - Country

population_file = "../Resources/population_GDP.csv"
population_data_df = pd.read_csv(population_file)
population_data = pd.DataFrame(population_data_df)
population_data.head()

,Country,Code,Population,GDP per Capita
0,Afghanistan,AFG,"32,526,562.00",594.323081
1,Albania,ALB,"2,889,167.00",3945.217582
2,Algeria,ALG,"39,666,519.00",4206.031232
3,American Samoa*,ASA,"55,538.00",NaN
4,Andorra,AND,"70,473.00",NaN


In [10]:
#create Athlete_event

athlete_event= athlete_data_df[['Medal']]

athlete_event = athlete_event.rename(columns={'Medal':'medal'})

In [13]:
athlete_event = athlete_event.dropna()

athlete_event

,medal
3,Gold
37,Bronze
38,Bronze
40,Bronze
41,Bronze
...,...
271078,Silver
271080,Bronze
271082,Bronze
271102,Bronze


In [14]:
athlete_event.drop_duplicates('medal', inplace=True)

athlete_event

,medal
3,Gold
37,Bronze
63,Silver


In [24]:
#create country df

country_pd = population_data[['Code','Country']]

country_rename = country_pd.rename(columns={'Code':'country_code', 'Country':'country'})

country_rename['id'] = ""

country_rename['id'] = country_rename['id'].index + 1

country = country_rename[['id','country_code','country']]

country

,id,country_code,country
0,1,AFG,Afghanistan
1,2,ALB,Albania
2,3,ALG,Algeria
3,4,ASA,American Samoa*
4,5,AND,Andorra
...,...,...,...
196,197,VIE,Vietnam
197,198,ISV,Virgin Islands*
198,199,YEM,Yemen
199,200,ZAM,Zambia


In [25]:
country.to_csv('country.csv')

In [ ]:
#create country_stat

country_stat = population_data[[' Population ','GDP per Capita']]

country_stat_rename = country_stat.rename(columns={' Population ': 'population', 'GDP per Capita':'gdp'})

country_stat_rename.set_index('population')

In [ ]:
country_stat_rename["year"] = "2014"
country_stat_rename["country_id"] = ""

In [ ]:
country_stats = country_stat_rename[["year","population", "gdp"]] 

country_stats.set_index('year')


country_stats['population'] = country_stats['population'].str.strip()

country_stats


In [ ]:
country_stats['population'] = country_stats['population'].str.replace(",","")

country_stats

In [ ]:
country_stats['population'] = country_stats['population'].str.replace(".00","")

country_stats_update = country_stats.fillna(0)


#use this one

country_stats_update

In [ ]:
#country_stats_update.astype(int)

In [ ]:
#country_stats_update.info()

In [15]:
#connect to db
rds_connection_string = f'{username}:{password}@localhost:5432/Olympics_db'
engine = create_engine(f'postgresql://{rds_connection_string}')

In [16]:
engine.table_names()

['country',
 'athlete',
 'event',
 'sport',
 'olympic_season',
 'athlete_event',
 'country_stats']

In [19]:
country.to_sql(name='country', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from country', con=engine).head()

In [ ]:
country_stats_update.to_sql(name='country_stats', con=engine, if_exists='append', index=False)

In [ ]:
pd.read_sql_query('select * from country_stats', con=engine).head()

In [17]:
athlete_event.to_sql(name='athlete_event', con=engine, if_exists='append', index=False)